How to switch between different config ?

Can we train multiple lora_adapter? for same task? different task parallely? 

from a list of adapters how to select? or load




In [2]:
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification, AutoTokenizer

from peft import LoraConfig, get_peft_model

In [3]:
peft_model_name = 'roberta-base-peft'

base_model = 'roberta-base'

In [4]:
##
model=AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=base_model)
model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [8]:
model_total_parameters=sum(p.numel() for p in model.parameters())
model_parameters_trainable=sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'total parameters: {model_total_parameters} | total_trainable_parameters={model_parameters_trainable}')

total parameters: 124647170 | total_trainable_parameters=124647170


In [11]:
##loading the peft model
peft_config=LoraConfig(inference_mode=False)

peft_model=get_peft_model(model=model,
                          peft_config=peft_config)

In [14]:
peft_model.print_trainable_parameters() ## only trains 23% of the model

trainable params: 294,912 || all params: 124,942,082 || trainable%: 0.2360389672392365


In [16]:
peft_model

PeftModel(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                    )
                    (lora_A): ModuleDict(
                      (default): Linear

Adding a adapter works same as adding adapter to peft_model from the hub. Basically we will register numbers of adapters tp the config. This does not change the model itself. While training and inference we can switch to any of them using different methods as explained below. More detail on previous colab note 09_PEFT.ipynb

In [29]:


peft_model.add_adapter('adapter1',peft_config)

In [30]:
peft_model

PeftModel(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                      (adapter1): Identity()
                      (adapter2): Identity()
             

In [31]:
peft_model.print_trainable_parameters()

trainable params: 294,912 || all params: 125,826,818 || trainable%: 0.23437928788757895


In [32]:
peft_model.add_adapter('adapter2',peft_config)
peft_model.add_adapter('adapter3',peft_config)

In [33]:
peft_model

PeftModel(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 768, padding_idx=1)
          (position_embeddings): Embedding(514, 768, padding_idx=1)
          (token_type_embeddings): Embedding(1, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-11): 12 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=768, out_features=768, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Identity()
                      (adapter1): Identity()
                      (adapter2): Identity()
             

In [34]:
peft_model.active_adapters

['adapter1']

In [39]:
peft_model.set_adapter('adapter1')  #We can use this method to see the active peft adapter

### peft_model.set_active_adapter

In [41]:
peft_model.set_adapter('adapter1')

In [42]:
peft_model.train_adapter('adapter1')

AttributeError: 'RobertaForSequenceClassification' object has no attribute 'train_adapter'